# Recommender System - Tensor Flow

## 1. Prerequisites

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings


warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.model_selection import train_test_split

import tensorflow as tf
import scipy
import scipy.io
import scipy.sparse as sp

from keras.layers import Input, Embedding, Flatten, Dot, Dense
from keras.models import Model

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from helpers import*
from preprocessing import*

In [27]:
DATA_TRAIN_PATH="data/data_train.csv"
ratings=load_data(DATA_TRAIN_PATH)

number of items: 10000, number of users: 1000


In [29]:
DATA_TEST_PATH = "data/sampleSubmission.csv"
sample = load_data(DATA_TEST_PATH)

number of items: 10000, number of users: 1000


In [30]:
data, n_users, n_movies = convert(ratings)

train, test = train_test_split(data, test_size=0.1, random_state=42)

## 2. Creating a dot product model

In [31]:
movie_input = Input(shape=[1], name="Movies-Input")
movie_embedding = Embedding(n_movies+1, 5, name="Movies-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Movies")(movie_embedding)

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

prod = Dot(name="Dot-Product", axes=1)([movie_vec, user_vec])
model1 = Model([user_input, movie_input], prod)
model1.compile('adam', 'mean_squared_error')

In [ ]:
from keras.models import load_model

if os.path.exists('regression_model.h5'):
    model1 = load_model('regression_model.h5')
else:
    history = model1.fit([train.user_id, train.movie_id], train.rating, epochs=20, verbose=1)
    model1.save('regression_model.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

Epoch 1/20
1059256/1059256 [==============================] - 48s 46us/step - loss: 1.0484
Epoch 2/20
1059256/1059256 [==============================] - 48s 45us/step - loss: 1.0435
Epoch 3/20
1059256/1059256 [==============================] - 49s 46us/step - loss: 1.0385
Epoch 4/20
1059256/1059256 [==============================] - 47s 44us/step - loss: 1.0308
Epoch 5/20
1059256/1059256 [==============================] - 51s 48us/step - loss: 1.0206
Epoch 6/20
1059256/1059256 [==============================] - 51s 48us/step - loss: 1.0086
Epoch 7/20
1059256/1059256 [==============================] - 49s 47us/step - loss: 0.9966
Epoch 8/20
1059256/1059256 [==============================] - 49s 46us/step - loss: 0.9850
Epoch 9/20
1059256/1059256 [==============================] - 50s 47us/step - loss: 0.9753
Epoch 10/20
1059256/1059256 [==============================] - 51s 48us/step - loss: 0.9677
Epoch 11/20
1059256/1059256 [==============================] - 51s 48us/step - loss: 0.96

In [24]:
model1.evaluate([test.user_id, test.movie_id], test.rating)

117696/117696 [==============================] - 2s 13us/step


1.0578510254102922

In [23]:
predictions = model1.predict([test.user_id, test.movie_id])